In [20]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler

In [21]:
NUMBER_OF_CLASSES=5
BATCH_SIZE=32 # using minibatch of 32 samples

In [22]:
template_features = np.genfromtxt('../data/interim/template_matching.csv', skip_header=True, dtype=float, delimiter=',')
gabriel_features = np.genfromtxt('../data/processed/combined_features.csv', skip_header=True, dtype=float, delimiter=',')
print('labels do match?', all(template_features[:, 0] == gabriel_features[:, 0]))
y = tf.one_hot(indices=template_features[:,0], depth=NUMBER_OF_CLASSES).numpy()
X = np.hstack((template_features[:,1:], gabriel_features[:,1:]))

X = X / X.max(axis=0) # alongside rows, per column

print(X.shape, y.shape)
number_of_features = X.shape[1]


labels do match? True
(10000, 121) (10000, 5)


In [23]:
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

Big question:
* Is it worthwhile to center data?
* Is it worthwhile to scale data or normalize to (0, 1) range?
For later!


EDIT:
First answer: FOR RELU YOU NEED TO SCALE FOR (0,1) AND FOR TANH FOR (-1,1) OTHERWISE IT WON'T WORK!

In [24]:
input_shape = (number_of_features,) # need to pass that comma, as otherwise it'd be evaluated as simple int i.e. type((5))==int, type((4,))==tuple
model = tf.keras.Sequential(
    [
    tf.keras.Input(shape=input_shape),
       tf.keras.layers.Dense(32, activation='relu'),
       tf.keras.layers.Dense(32, activation='relu'),
       tf.keras.layers.Dense(32, activation='relu'),
       tf.keras.layers.Dense(units=NUMBER_OF_CLASSES, activation='softmax')
    ]
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 32)             │         3,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,181 (24.14 KB)

 Trainable params: 6,181 (24.14 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), # stochastic gradient descent, but can specify batch size.
    loss='categorical_crossentropy',
    metrics=['AUC'],
)

In [26]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - AUC: 0.5719 - loss: 1.5970
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.8271 - loss: 1.4232
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.8930 - loss: 1.0842
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.9312 - loss: 0.8331
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.9494 - loss: 0.6716
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.9546 - loss: 0.6131
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.9622 - loss: 0.5523
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.9650 - loss: 0.5278
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.9667 - loss: 0.5097
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - AUC: 0.9717 - loss: 0.4718


In [27]:
results = model.evaluate(X_val, y_val, batch_size=32)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - AUC: 0.9684 - loss: 0.4990
